<a href="https://www.kaggle.com/code/namitaachyuthanpesu/boostisthesecretofmyenergy?scriptVersionId=208541750" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-politics-2004-2019/IndiaVotes_PC__All_States_2009.csv
/kaggle/input/indian-politics-2004-2019/IndiaVotes_PC__All_States_2014.csv
/kaggle/input/indian-politics-2004-2019/2024.csv
/kaggle/input/indian-politics-2004-2019/2019.csv
/kaggle/input/indian-politics-2004-2019/cleaned.csv
/kaggle/input/indian-politics-2004-2019/2004.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import optuna
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

In [4]:
df = pd.read_csv('/kaggle/input/indian-politics-2004-2019/cleaned.csv')
df.head()

,Unnamed: 0,State_2004,Party_2004,Electors_2004,Votes_2004,Turnout_2004,Margin_2004,Margin %_2004,State_2009,Party_2009,...,Turnout_2019,Margin_2019,Margin %_2019,State_2024,Party_2024,Electors_2024,Votes_2024,Turnout_2024,Margin_2024,Margin %_2024
0,Adilabad,0,0,1140834.0,830601.0,72.8,40974.0,4.9,0,28,...,77.9,58560.0,5.5,31,2,1650175.0,1235597.0,74.9,90652.0,7.3
1,Agra,1,1,1430713.0,642675.0,44.9,57342.0,8.9,1,2,...,61.7,211546.0,18.5,1,2,2072685.0,1123779.0,54.2,271294.0,24.1
2,Ajmer,2,2,1203803.0,529138.0,44.0,127976.0,24.2,2,4,...,69.0,416424.0,33.0,2,2,1995699.0,1201179.0,60.2,329991.0,27.5
3,Akbarpur,1,3,1446088.0,741543.0,51.3,58269.0,7.9,1,4,...,60.3,275142.0,26.8,1,2,1869167.0,1086953.0,58.2,44345.0,4.1
4,Akola,3,2,1271055.0,734524.0,57.8,106371.0,14.5,3,2,...,64.2,275596.0,24.6,3,2,1890814.0,1173072.0,62.0,40626.0,3.5


# 2019

In [5]:
# Step 1: Get the unique classes and sort them
original_classes = sorted(df['Party_2019'].unique())

# Step 2: Create a mapping from original classes to new labels starting from 0
label_mapping = {original_classes[i]: i for i in range(len(original_classes))}

# Step 3: Map the original labels in the DataFrame to the new labels
df['Party_2019_mapped'] = df['Party_2019'].map(label_mapping)

# Display the mapping and the new class labels
print("Original classes:", original_classes)
print("Label mapping:", label_mapping)
print("New class labels:", df['Party_2019_mapped'].unique())

Original classes: [0, 1, 2, 3, 4, 6, 7, 8, 12, 15, 17, 18, 19, 21, 22, 23, 26, 32, 34, 39, 41, 42, 46, 47, 49, 51, 52, 53, 54, 55, 56, 57]
Label mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7, 12: 8, 15: 9, 17: 10, 18: 11, 19: 12, 21: 13, 22: 14, 23: 15, 26: 16, 32: 17, 34: 18, 39: 19, 41: 20, 42: 21, 46: 22, 47: 23, 49: 24, 51: 25, 52: 26, 53: 27, 54: 28, 55: 29, 56: 30, 57: 31}
New class labels: [ 2  6  4  3 12 17  8  1  7  9  5 18 25 20 26 15 11 27  0 19 28 16 29 10
 30 22 23 13 21 31 24 14]


In [6]:
features = [
    'State_2004', 'Electors_2004', 'Votes_2004', 'Turnout_2004', 'Margin_2004', 'Margin %_2004',
    'State_2009', 'Electors_2009', 'Votes_2009', 'Turnout_2009', 'Margin_2009', 'Margin %_2009',
    'State_2014', 'Electors_2014', 'Votes_2014', 'Turnout_2014', 'Margin_2014', 'Margin %_2014'
]
target = 'Party_2019_mapped'

In [7]:
X = df[features]
y = df[target]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Ensemble

In [9]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
#lgb_model = LGBMClassifier(silent = True)
cat_model = CatBoostClassifier(silent=True)

In [10]:
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_model),
        # ('lgb', lgb_model),
        ('cat', cat_model)
    ],
    voting='soft'
)

In [11]:
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learni...ne, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7df7da49b1f0>)],
                 voting='soft')

In [12]:
accuracy = voting_classifier.score(X_test, y_test)
print(f'Ensemble Model Accuracy: {accuracy:.2f}')

Ensemble Model Accuracy: 0.66


# 2024

In [13]:
original_classes_2024 = sorted(df['Party_2024'].unique())

label_mapping_2024 = {original_classes_2024[i]: i for i in range(len(original_classes_2024))}

df['Party_2024_mapped'] = df['Party_2024'].map(label_mapping_2024)

print("Original classes for 2024:", original_classes_2024)
print("Label mapping for 2024:", label_mapping_2024)
print("New class labels for 2024:", df['Party_2024_mapped'].unique())

Original classes for 2024: [1, 2, 4, 6, 7, 8, 11, 13, 17, 18, 21, 22, 25, 29, 32, 34, 42, 47, 51, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
Label mapping for 2024: {1: 0, 2: 1, 4: 2, 6: 3, 7: 4, 8: 5, 11: 6, 13: 7, 17: 8, 18: 9, 21: 10, 22: 11, 25: 12, 29: 13, 32: 14, 34: 15, 42: 16, 47: 17, 51: 18, 53: 19, 54: 20, 56: 21, 57: 22, 58: 23, 59: 24, 60: 25, 61: 26, 62: 27, 63: 28, 64: 29, 65: 30, 66: 31, 67: 32, 68: 33, 69: 34}
New class labels for 2024: [ 1  2  0 23 14 24  7  5 25 26 27  3  6 15 18 10  4 28 29 30 31 16 19 32
  9 12 20 33  8 21 17 34 22 11 13]


In [14]:
features_2024 = [
    'State_2004', 'Electors_2004', 'Votes_2004', 'Turnout_2004', 'Margin_2004', 'Margin %_2004',
    'State_2009', 'Electors_2009', 'Votes_2009', 'Turnout_2009', 'Margin_2009', 'Margin %_2009',
    'State_2014', 'Electors_2014', 'Votes_2014', 'Turnout_2014', 'Margin_2014', 'Margin %_2014',
    'State_2019', 'Electors_2019', 'Votes_2019', 'Turnout_2019', 'Margin_2019', 'Margin %_2019'
]

target_2024 = 'Party_2024_mapped'

X = df[features_2024]
y = df[target_2024]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
#lgb_model = LGBMClassifier(silent = True)
cat_model = CatBoostClassifier(silent=True)

In [17]:
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_model),
        # ('lgb', lgb_model),
        ('cat', cat_model)
    ],
    voting='soft'
)

In [18]:
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learni...ne, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7df78b643c70>)],
                 voting='soft')

In [19]:
accuracy = voting_classifier.score(X_test, y_test)
print(f'Ensemble Model Accuracy: {accuracy:.2f}')

Ensemble Model Accuracy: 0.58
